In [1]:
import hydra
from omegaconf import OmegaConf
from glob import  glob
from rosa import  predict
from rosa.data import RosaDataModule, create_io_paths
from rosa.utils import RosaConfig
from rosa.modeling.modules import RosaLightningModule


BASE_DIR = "/home/ec2-user/outputs/2023-03-03/07-28-03"
config_dir = BASE_DIR + "/.hydra"

with hydra.initialize_config_dir(config_dir=config_dir):
    config = hydra.compose(config_name="config", overrides=OmegaConf.load(config_dir + "/overrides.yaml"))

    _, output_path = create_io_paths(config.paths)
    config.data_module.batch_size = 12
    config.module.model.layer_norm_1 = None
    config.module.model.input_embed.embedding_dim = config.module.model.transformer.dim
    config.module.model.input_embed_1.embedding_dim = config.module.model.transformer.dim
    # config.module.model.transformer = None
    # Create Data Module
    rdm = RosaDataModule(
        output_path,
        config=config.data_module,
    )
    rdm.setup()

    # Load model from checkpoint
    rlm = RosaLightningModule(
        in_dim=rdm.len_input,
        out_dim=rdm.len_target,
        config=config.module,
        var_input=rdm.predict_dataset.input[1]
    )
    print(rlm)

Global seed set to 0
/tmp/ipykernel_29353/3364748425.py:13: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize_config_dir(config_dir=config_dir):


RosaLightningModule(
  (model): RosaFormerModel(
    (main): Sequential(
      (dual_embed): ParallelEmbed(
        (models): ModuleList(
          (0): Sequential(
            (input_embed_0): MaskedEmbed(
              (mask): Linear(in_features=1, out_features=256, bias=True)
              (model): Sequential(
                (projection): Linear(in_features=1, out_features=256, bias=True)
                (layer_norm): Identity()
                (dropout): Dropout(p=0, inplace=False)
              )
            )
          )
          (1): Sequential(
            (layer_norm_1): Identity()
            (input_embed_1): InputEmbed(
              (model): Sequential(
                (projection): Linear(in_features=3072, out_features=256, bias=True)
                (layer_norm): Identity()
                (dropout): Dropout(p=0, inplace=False)
              )
            )
          )
        )
      )
      (join_embeds): AddEmbeds()
      (transformer): Performer(
        (net): Sequ

In [26]:
config.data_module.batch_size

10

In [2]:
from torch.profiler import profile, record_function, ProfilerActivity

In [16]:
with profile(activities=[ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof:
    batch = next(iter(rdm.train_dataloader()))
    rlm.transfer_batch_to_device(batch, 'cuda', 0)


STAGE:2023-03-08 01:18:54 13436:13436 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-08 01:18:54 13436:13436 ActivityProfilerController.cpp:300] Completed Stage: Collection
[W collection.cpp:634] Warning: Failed to recover relationship between all profiler and kineto events: 128 vs. 0  reassociated. (function reassociate)


In [19]:
print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=10, top_level_events_only=True))


This report only display top-level ops statistics
------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls  
------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            [memory]         0.00%       0.000us         0.00%       0.000us       0.000us       0.000us         0.00%       0.000us       0.000us           0 b           0 b           128  
               cudaStreamIsCapturing         0.45%      11.000us         0.45%      11.000us       5.500us       0.000us 

In [30]:
import torch.profiler as profiler


with profiler.profile(# schedule=profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
    on_trace_ready=profiler.tensorboard_trace_handler('/home/ec2-user/rosa/notebooks/log/ex0'),
    record_shapes=True,
    profile_memory=True,
    with_stack=True
) as prof:
    rdm.predict_dataset.input[1].to('cuda')
    prof.step()
    for i, batch in enumerate(iter(rdm.train_dataloader())):
        rlm.transfer_batch_to_device(batch, 'cuda', i)
        prof.step()
    # batch = next(iter(rdm.train_dataloader()))
    # rlm.transfer_batch_to_device(batch, 'cuda', 0)
    # prof.step()
    

STAGE:2023-03-08 04:32:19 17149:17149 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-08 04:32:19 17149:17149 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-08 04:32:19 17149:17149 output_json.cpp:417] Completed Stage: Post Processing


In [3]:
import torch.profiler as profiler


with profiler.profile(# schedule=profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
    on_trace_ready=profiler.tensorboard_trace_handler('/home/ec2-user/rosa/notebooks/log/ex8'),
    record_shapes=True,
    profile_memory=True,
    with_stack=True
) as prof:
    rlm.model.to('cuda')
    prof.step()
    for i, batch in enumerate(iter(rdm.train_dataloader())):
        batch = rlm.transfer_batch_to_device(batch, 'cuda', i)
        rlm.training_step(batch, i)
        prof.step()
    # batch = next(iter(rdm.train_dataloader()))
    # rlm.transfer_batch_to_device(batch, 'cuda', 0)
    # prof.step()
    

STAGE:2023-03-08 04:32:53 29353:29353 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
[W CPUAllocator.cpp:231] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
/opt/conda/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:416: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(
STAGE:2023-03-08 04:32:55 29353:29353 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-03-08 04:32:57 29353:29353 output_json.cpp:417] Completed Stage: Post Processing


In [4]:
import  torch

torch.cuda.memory_stats()

OrderedDict([('active.all.allocated', 1316),
             ('active.all.current', 48),
             ('active.all.freed', 1268),
             ('active.all.peak', 105),
             ('active.large_pool.allocated', 842),
             ('active.large_pool.current', 3),
             ('active.large_pool.freed', 839),
             ('active.large_pool.peak', 46),
             ('active.small_pool.allocated', 474),
             ('active.small_pool.current', 45),
             ('active.small_pool.freed', 429),
             ('active.small_pool.peak', 60),
             ('active_bytes.all.allocated', 244465911296),
             ('active_bytes.all.current', 251234816),
             ('active_bytes.all.freed', 244214676480),
             ('active_bytes.all.peak', 14649931264),
             ('active_bytes.large_pool.allocated', 244278289408),
             ('active_bytes.large_pool.current', 243421696),
             ('active_bytes.large_pool.freed', 244034867712),
             ('active_bytes.large_pool.peak

In [12]:
model = rlm.model
param_size = 0
n_param = 0
for param in model.parameters():
    n_param += param.nelement()
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.2f}MB'.format(size_all_mb))
print('number of parameters {:.2f}M'.format(n_param / 1e6))

model size: 9.18MB
number of parameters 2.37M


In [14]:
import torch

torch.cuda.memory_allocated() / 1e6

248.707072

In [61]:
rlm.model.to('cuda')

RosaFormerModel(
  (main): Sequential(
    (dual_embed): ParallelEmbed(
      (models): ModuleList(
        (0): Sequential(
          (input_embed_0): MaskedEmbed(
            (mask): Linear(in_features=1, out_features=256, bias=True)
            (model): Sequential(
              (projection): Linear(in_features=1, out_features=256, bias=True)
              (layer_norm): Identity()
              (dropout): Dropout(p=0, inplace=False)
            )
          )
        )
        (1): Sequential(
          (layer_norm_1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
          (input_embed_1): InputEmbed(
            (model): Sequential(
              (projection): Linear(in_features=3072, out_features=256, bias=True)
              (layer_norm): Identity()
              (dropout): Dropout(p=0, inplace=False)
            )
          )
        )
      )
    )
    (join_embeds): AddEmbeds()
    (transformer): Performer(
      (net): SequentialSequence(
        (layers): ModuleList

In [59]:
rlm.model(batch[0])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_mm)

In [51]:
batch

[[[tensor([[[7.1979e-02],
            [2.4573e-02],
            [1.6449e-02],
            ...,
            [0.0000e+00],
            [0.0000e+00],
            [0.0000e+00]],
   
           [[2.0022e+00],
            [8.7149e-01],
            [7.6668e-01],
            ...,
            [2.5435e-04],
            [0.0000e+00],
            [0.0000e+00]],
   
           [[1.4324e-02],
            [3.0532e-01],
            [4.2370e-02],
            ...,
            [0.0000e+00],
            [0.0000e+00],
            [0.0000e+00]],
   
           ...,
   
           [[1.2376e+00],
            [4.0734e-01],
            [8.2620e-02],
            ...,
            [7.7564e-04],
            [7.7564e-04],
            [0.0000e+00]],
   
           [[1.1483e-01],
            [6.7977e-01],
            [3.1118e-01],
            ...,
            [0.0000e+00],
            [0.0000e+00],
            [0.0000e+00]],
   
           [[4.3647e-01],
            [6.5550e-01],
            [4.1233e-02],
            

In [21]:
# !pip install torch-tb-profiler

In [40]:
rdm.predict_dataset.input[1].shape

torch.Size([19431, 3072])

In [42]:
19431 * 3072 / 1e6

59.692032

In [6]:
batch = next(iter(rdm.train_dataloader()))

In [7]:
batch

[[[tensor([[1.2949e+00, 6.6912e-01, 2.5456e-01,  ..., 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [1.1483e-01, 6.7977e-01, 3.1118e-01,  ..., 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [4.6344e-02, 8.1008e-01, 3.9162e-01,  ..., 5.9092e-04, 0.0000e+00,
            0.0000e+00],
           ...,
           [1.1423e+00, 4.2181e-01, 1.6588e-01,  ..., 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [3.1224e+00, 2.9315e-01, 4.1616e-01,  ..., 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [3.9322e+00, 1.6058e+00, 1.5700e-01,  ..., 0.0000e+00, 0.0000e+00,
            0.0000e+00]]),
   tensor([[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False,  True,  True],
           [ True, False,  True,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False,  True,  ..., False, False, False],
           [False, False,  True,  ...,  Tru

In [6]:
config.module.model.layer_norm_1

True

In [9]:
config.module.model.transformer = None